In [1]:
# from matmul_had import *
# from utils import *
import os

# 设置只使用特定的 GPU（例如 GPU 0 和 GPU 1）
os.environ['CUDA_VISIBLE_DEVICES'] = '4,5,6,7'

# 检查设置是否生效
print("CUDA_VISIBLE_DEVICES =", os.environ['CUDA_VISIBLE_DEVICES'])
import argparse
import os 
import datetime
import numpy as np
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM
from importlib.metadata import version
print('torch', version('torch'))

from lib.utils import find_layers
from lib.data import prepare_calibration_input
from lib.data import get_loaders 
from lib.eval import eval_ppl, eval_zero_shot
from lib.esd_utils import get_esd_metrics

from main import get_llm

from tqdm import tqdm

print('torch', version('torch'))
print('transformers', version('transformers'))
print('accelerate', version('accelerate'))
print('# of gpus: ', torch.cuda.device_count())

class PruneConfig:
    def __init__(self):
        self.model = '/h3cstore_ns/jcxie/hf_weights/llama-2-13b-hf'
        self.seed = 0
        self.nsamples = 128
        self.sparsity_ratio = 0.7
        self.sparsity_type = "unstructured"
        self.prune_method = "sparsegpt_silu_ww"
        self.cache_dir = "llm_weights"
        self.use_variant = False
        self.save = '/h3cstore_ns/jcxie/LISA/wanda-main/ckpt'
        self.save_model = None
        self.exclude = 'gate_proj'
        self.ww_metric = "alpha_peak"
        self.ww_metric_cache = "/h3cstore_ns/jcxie/LISA/wanda-main/data/llama2-7b-hf"
        self.epsilon = 0.3
        self.mapping_type = "block_wise"
        self.Hyper_m = 3.0
        self.Lamda = 0.20
        self.eval_zero_shot = False
args = PruneConfig()
model = get_llm(args.model, args.cache_dir)
model.eval()
for n, p in model.named_parameters():
    print(n, p.size())
tokenizer = AutoTokenizer.from_pretrained(args.model, use_fast=False)

with torch.no_grad():
    use_cache = model.config.use_cache 
    model.config.use_cache = False 
    ratios = None
    device = model.hf_device_map["lm_head"]

    # print("loading calibdation data")
    # dataloader, _ = get_loaders("c4",nsamples=args.nsamples,seed=args.seed,seqlen=model.seqlen,tokenizer=tokenizer)
    # print("dataset loading complete")
    # with torch.no_grad():
    #     inps, outs, attention_mask, position_ids = prepare_calibration_input(model, dataloader, device, nsamples=args.nsamples)

    layers = model.model.layers

    layer_num = len(find_layers(layers))
    if ratios is None:
        ratios = [args.sparsity_ratio for i in range(layer_num)]
    k=0


from torch import nn
with torch.no_grad():
    print('Starting ...')
    dataloader, _ = get_loaders(args, "c4",nsamples=args.nsamples,seed=args.seed,seqlen=model.seqlen,tokenizer=tokenizer)

    use_cache = model.config.use_cache
    model.config.use_cache = False
    layers = model.model.layers

    if "model.embed_tokens" in model.hf_device_map:
        dev = model.hf_device_map["model.embed_tokens"]

    dtype = next(iter(model.parameters())).dtype
    inps = torch.zeros(
        (args.nsamples, model.seqlen, model.config.hidden_size), dtype=dtype, device=dev
    )
    cache = {'i': 0, 'attention_mask': None, "position_ids": None}

    class Catcher(nn.Module):
        def __init__(self, module):
            super().__init__()
            self.module = module
        def forward(self, inp, **kwargs):
            inps[cache['i']] = inp
            cache['i'] += 1
            cache['attention_mask'] = kwargs['attention_mask']
            cache['position_ids'] = kwargs['position_ids']
            raise ValueError
    layers[0] = Catcher(layers[0])
    for batch in dataloader:
        try:
            model(batch[0].to(dev))
        except ValueError:
            pass
    layers[0] = layers[0].module
    torch.cuda.empty_cache()

    outs = torch.zeros_like(inps)
    attention_mask = cache['attention_mask']
    position_ids = cache['position_ids']

    print('Ready.')


CUDA_VISIBLE_DEVICES = 4,5,6,7
torch 2.3.1+cu121
torch 2.3.1+cu121
transformers 4.47.1
accelerate 0.29.1
# of gpus:  4
torch 2.3.1+cu121
transformers 4.47.1
accelerate 0.29.1
# of gpus:  4


2025-05-25 17:00:15.159836: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2025-05-25 17:00:15.319573: I tensorflow/core/util/port.cc:104] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-05-25 17:00:15.324427: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/cuda-12.3/lib64:/usr/local/nvidia/lib:/usr/local/nvidia/lib64
2025-05-25

Loading checkpoint shards:   0%|          | 0/6 [00:00<?, ?it/s]

model.embed_tokens.weight torch.Size([32000, 5120])
model.layers.0.self_attn.q_proj.weight torch.Size([5120, 5120])
model.layers.0.self_attn.k_proj.weight torch.Size([5120, 5120])
model.layers.0.self_attn.v_proj.weight torch.Size([5120, 5120])
model.layers.0.self_attn.o_proj.weight torch.Size([5120, 5120])
model.layers.0.mlp.gate_proj.weight torch.Size([13824, 5120])
model.layers.0.mlp.up_proj.weight torch.Size([13824, 5120])
model.layers.0.mlp.down_proj.weight torch.Size([5120, 13824])
model.layers.0.input_layernorm.weight torch.Size([5120])
model.layers.0.post_attention_layernorm.weight torch.Size([5120])
model.layers.1.self_attn.q_proj.weight torch.Size([5120, 5120])
model.layers.1.self_attn.k_proj.weight torch.Size([5120, 5120])
model.layers.1.self_attn.v_proj.weight torch.Size([5120, 5120])
model.layers.1.self_attn.o_proj.weight torch.Size([5120, 5120])
model.layers.1.mlp.gate_proj.weight torch.Size([13824, 5120])
model.layers.1.mlp.up_proj.weight torch.Size([13824, 5120])
model.l

normalizer.cc(51) LOG(INFO) precompiled_charsmap is empty. use identity normalization.
100%|██████████| 128/128 [00:50<00:00,  2.55it/s]


Ready.


In [2]:
prune_n, prune_m = 0, 0
if args.sparsity_type != "unstructured":
    assert args.sparsity_ratio == 0.5, "sparsity ratio must be 0.5 for structured N:M sparsity"
    prune_n, prune_m = map(int, args.sparsity_type.split(":"))
    
from lib.sparsegpt import SparseGPTV2, SparseGPT
import numpy as np
with torch.no_grad():
    
    layer_num = len(find_layers(layers))
    if ratios is None:
        ratios = [args.sparsity_ratio for i in range(layer_num)]
    k=0

    for i in range(len(layers)):
        layer = layers[i]
        if f"model.layers.{i}" in model.hf_device_map:
            dev = model.hf_device_map[f"model.layers.{i}"]
            print(f"layer {i} device {dev}")
            if attention_mask is None:
                inps, outs, position_ids = inps.to(dev), outs.to(dev), position_ids.to(dev)
            else:
                inps, outs, attention_mask, position_ids = inps.to(dev), outs.to(dev), attention_mask.to(dev), position_ids.to(dev)

        subset = find_layers(layer)

        gpts = {}
        for name in subset:

            if name == 'self_attn.v_proj' and i == 8:
                gpts[name] = SparseGPTV2(subset[name], have_scaler=True)
            else:
                gpts[name] = SparseGPT(subset[name])

        def add_batch(name):
            def tmp(_, inp, out):
                gpts[name].add_batch(inp[0].data, out.data)
            return tmp

        handles = []
        for name in gpts:
            handles.append(subset[name].register_forward_hook(add_batch(name)))

        for j in range(args.nsamples):
                if attention_mask is None:
                    outs[j] = layer(inps[j].unsqueeze(0), position_ids=position_ids)[0]
                else:
                    outs[j] = layer(inps[j].unsqueeze(0), attention_mask=attention_mask, position_ids=position_ids)[0]
        for h in handles:
            h.remove()

        if i == 8:
            test_inps = gpts['self_attn.v_proj'].inps
            test_outs = gpts['self_attn.v_proj'].outs
            ora_W = model.model.layers[i].self_attn.v_proj.weight.data
            # np.save('/h3cstore_ns/jcxie/LISA/wanda-main/npys/test_admm/ora_W.npy', ora_W.cpu().numpy())

        for name in gpts:
            print(i, name)
            print('Pruning ...')

            gpts[name].fasterprune(ratios[k], prune_n=prune_n, prune_m=prune_m, percdamp=0.01, blocksize=128)
            gpts[name].free()
            k+=1
        
        if i == 8:
            new_W = gpts['self_attn.v_proj'].layer.weight.data
            scaler_row = gpts['self_attn.v_proj'].scaler_row
            # np.save('/h3cstore_ns/jcxie/LISA/wanda-main/npys/test_admm/new_W.npy', new_W.cpu().numpy())
            break

        for j in range(args.nsamples):
            if attention_mask is None:
                outs[j] = layer(inps[j].unsqueeze(0), position_ids=position_ids)[0]
            else:
                outs[j] = layer(inps[j].unsqueeze(0), attention_mask=attention_mask, position_ids=position_ids)[0]

        layers[i] = layer 
        torch.cuda.empty_cache()

        inps, outs = outs, inps

    model.config.use_cache = use_cache
    torch.cuda.empty_cache()

layer 0 device 0


The attention layers in this model are transitioning from computing the RoPE embeddings internally through `position_ids` (2D tensor with the indexes of the tokens), to using externally computed `position_embeddings` (Tuple of tensors, containing cos and sin). In v4.46 `position_ids` will be removed and `position_embeddings` will be mandatory.


0 self_attn.q_proj
Pruning ...
0 self_attn.k_proj
Pruning ...
0 self_attn.v_proj
Pruning ...
0 self_attn.o_proj
Pruning ...
0 mlp.gate_proj
Pruning ...
0 mlp.up_proj
Pruning ...
0 mlp.down_proj
Pruning ...
layer 1 device 0
1 self_attn.q_proj
Pruning ...
1 self_attn.k_proj
Pruning ...
1 self_attn.v_proj
Pruning ...
1 self_attn.o_proj
Pruning ...
1 mlp.gate_proj
Pruning ...
1 mlp.up_proj
Pruning ...
1 mlp.down_proj
Pruning ...
layer 2 device 0
2 self_attn.q_proj
Pruning ...
2 self_attn.k_proj
Pruning ...
2 self_attn.v_proj
Pruning ...
2 self_attn.o_proj
Pruning ...
2 mlp.gate_proj
Pruning ...
2 mlp.up_proj
Pruning ...
2 mlp.down_proj
Pruning ...
layer 3 device 0
3 self_attn.q_proj
Pruning ...
3 self_attn.k_proj
Pruning ...
3 self_attn.v_proj
Pruning ...
3 self_attn.o_proj
Pruning ...
3 mlp.gate_proj
Pruning ...
3 mlp.up_proj
Pruning ...
3 mlp.down_proj
Pruning ...
layer 4 device 0
4 self_attn.q_proj
Pruning ...
4 self_attn.k_proj
Pruning ...
4 self_attn.v_proj
Pruning ...
4 self_attn.o_p

In [3]:
test_inps = torch.stack(test_inps)
test_outs = torch.stack(test_outs)

In [4]:
print('ora_W', ora_W.shape)
print('new_W', new_W.shape)
print('test_inps', test_inps.shape)
print('test_outs', test_outs.shape)
import torch
ora_W = ora_W.cuda()
new_W = new_W.cuda()
test_inps = test_inps.cpu()
test_outs = test_outs.cpu()

ora_W torch.Size([5120, 5120])
new_W torch.Size([5120, 5120])
test_inps torch.Size([128, 5120, 4096])
test_outs torch.Size([128, 1, 5120, 4096])


In [5]:
import numpy as np
np.save('/h3cstore_ns/jcxie/LISA/nips2024/test_npy/9_self_attn.v_proj_test_inps.npy', test_inps.cpu().numpy())
print(1)
np.save('/h3cstore_ns/jcxie/LISA/nips2024/test_npy/9_self_attn.v_proj_test_outs.npy', test_outs.cpu().numpy())
print(2)
np.save('/h3cstore_ns/jcxie/LISA/nips2024/test_npy/9_self_attn.v_proj_ora_W.npy', ora_W.cpu().numpy())
print(3)
np.save('/h3cstore_ns/jcxie/LISA/nips2024/test_npy/9_self_attn.v_proj_new_W.npy', new_W.cpu().numpy())

1
2
3


In [1]:
import numpy as np
import torch
ora_W = np.load('/h3cstore_ns/jcxie/LISA/nips2024/test_npy/9_self_attn.v_proj_ora_W.npy')
new_W = np.load('/h3cstore_ns/jcxie/LISA/nips2024/test_npy/9_self_attn.v_proj_new_W.npy')
test_inps = np.load('/h3cstore_ns/jcxie/LISA/nips2024/test_npy/9_self_attn.v_proj_test_inps.npy')
test_outs = np.load('/h3cstore_ns/jcxie/LISA/nips2024/test_npy/9_self_attn.v_proj_test_outs.npy')

ora_W = torch.tensor(ora_W).to(torch.float16).cuda()
new_W = torch.tensor(new_W).to(torch.float16).cuda()
test_inps = torch.tensor(test_inps).to(torch.float16)
test_outs = torch.tensor(test_outs).to(torch.float16)

In [ ]:
from tqdm import tqdm
import math
import time

lambda_value = []
lambda_abs = []
alphas = []

out = []

def alpha_schedule_exp(k, max_iter=100, alpha_start=0.9, alpha_end=0.99, gamma=5.0):
    # if k >= max_iter:
    #     return alpha_end
    ratio = k / max_iter
    return alpha_end - (alpha_end - alpha_start) * np.exp(-gamma * ratio)

def delta_schedule_exp(k, max_iter=100, alpha_start=0.9, alpha_end=0.99, gamma=5.0):
    if k >= max_iter:
        return alpha_end - alpha_start
    else:
        return alpha_schedule_exp(k, max_iter, alpha_start, alpha_end, gamma) - alpha_start

def alpha_schedule(k, max_iter=100, alpha_start=0.9, alpha_end=0.99):
    if k >= max_iter:
        return alpha_end
    else:
        return alpha_start + (alpha_end - alpha_start) * (k / max_iter)

def dual_ascent_method10(H_A, H_B, W_old, M, beta, alpha, gama, rho, epsilon=1e-1, max_iter=10, lambda_zero=False, percdamp=.01):
    M = M.to(torch.float32)
    H_A = H_A.to(torch.float32)
    H_B = H_B.to(torch.float32)
    W_old = W_old.to(torch.float32)
    
    # 初始化 W 和 Lambda
    W = W_old.clone()
    if lambda_zero:
        Lambda = torch.zeros_like(W)
    # else:
    #     term1 = beta * (torch.mm(W, H_A) - H_B)
    #     term2 = alpha * (W - W_old)
    #     Lambda = -M * (term1 + term2)
    I = torch.eye(H_A.shape[0], device=H_A.device)
    A = (1-alpha) * H_A + alpha * I

    A_start = torch.linalg.cholesky(A)
    A_start = torch.cholesky_inverse(A_start)
    
    A1 = -(A_start @ (I - H_A) @ A_start)
    A2 = (-A1) @ (I - H_A) @ A_start
    A3 = (-A2) @ (I - H_A) @ A_start


    alpha_start = alpha
    for k in tqdm(range(max_iter)):
        alpha = alpha_schedule_exp(k, alpha_start=alpha_start)
        delta = alpha - alpha_start

        if k % 20 == 0 and k < 100:
            alpha_start = alpha

            A = (1-alpha) * H_A + alpha * I

            A_start = torch.linalg.cholesky(A)
            A_start = torch.cholesky_inverse(A_start)
            
            A1 = -(A_start @ (I - H_A) @ A_start)
            A2 = (-A1) @ (I - H_A) @ A_start
            A3 = (-A2) @ (I - H_A) @ A_start



        W_prev = W.clone()

        B = (1-alpha-delta) * H_B + (alpha + delta) * W_prev
        B = B - Lambda

        # A_inv = (1-alpha-delta) * H_A + (alpha + delta) * I
        # A_inv = torch.linalg.cholesky(A_inv)
        # A_inv = torch.cholesky_inverse(A_inv)

        W = B @ (A_start + delta * A1 + delta**2 * A2 + delta**3 * A3)
        # W = B @ A_inv

        # print("二阶近似误差",torch.norm(((A_start + delta * A1 + delta**2 * A2) - A_inv)))


        Lambda = Lambda + rho * (M * W)

        lambda_value.append(Lambda.mean().item())
        lambda_abs.append(Lambda.abs().mean().item())
        out.append((M * W).mean().item())
        # W = W - M * W
        # 收敛判断

        if k % 50 == 0 and k > 0:
            

            # print( (W.to(torch.float16) @ test_inps[0].cuda() - test_outs[0][0].cuda()).abs().max(), (W.to(torch.float16) @ test_inps[0].cuda() - test_outs[0][0].cuda()).abs().mean())
            # print(torch.norm(W - W_prev))
            if torch.norm(W - W_prev) < epsilon:
                print( (W.to(torch.float16) @ test_inps[0].cuda() - test_outs[0][0].cuda()).abs().max(), (W.to(torch.float16) @ test_inps[0].cuda() - test_outs[0][0].cuda()).abs().mean())
                print(f"Converged at iteration {k}")
                print(torch.norm(W - W_prev))
                break

        # if k % 10 == 0 :
        #     print(torch.norm(W - W_prev))
    W = W - M * W
    return W.to(torch.float16)

test_x = test_inps[0]
test_y = test_outs[0][0]

H_A = torch.zeros((test_x.shape[-2], test_x.shape[-2]), device="cuda")
H_B = torch.zeros((test_y.shape[-2], test_x.shape[-2]), device="cuda")
nsamples = 0
for i in range(120):
    H_A *= nsamples / (nsamples + 1)
    H_B *= nsamples / (nsamples + 1)

    nsamples += 1

    H_A += (math.sqrt(2 / nsamples) * test_inps[i].cuda().to(torch.float32)) @ (math.sqrt(2 / nsamples) * test_inps[i].cuda().to(torch.float32)).T
    H_B += (math.sqrt(2 / nsamples) * test_outs[i][0].cuda().to(torch.float32)) @ (math.sqrt(2 / nsamples) * test_inps[i].cuda().to(torch.float32)).T

    

M = (new_W == 0).to(torch.float32).to(H_B.device)
print(M.device)
# M = torch.zeros_like(M).to(torch.float32).cuda()
print((M == 0).sum() / M.numel())
W_old = new_W.clone().to(H_B.device)
beta = 0.01
alpha = 0.9
gama = 0.0000
rho = 1


start = time.time()

W_update = dual_ascent_method10(H_A, H_B, W_old, M, beta, alpha, gama, rho, lambda_zero=True, max_iter=1000)
print('time', time.time() - start)

In [40]:
from tqdm import tqdm
import math
import time

lambda_value = []
lambda_abs = []
alphas = []

out = []

def alpha_schedule_exp(k, max_iter=100, alpha_start=0.5, alpha_end=0.99, gamma=5.0):
    # if k >= max_iter:
    #     return alpha_end
    ratio = k / max_iter
    return alpha_end - (alpha_end - alpha_start) * np.exp(-gamma * ratio)

def alpha_schedule(k, max_iter=100, alpha_start=0.9, alpha_end=0.99):
    if k >= max_iter:
        return alpha_end
    else:
        return alpha_start + (alpha_end - alpha_start) * (k / max_iter)

def dual_ascent_method8(H_A, H_B, W_old, M, beta, alpha, gama, rho, epsilon=1e-1, max_iter=10, lambda_zero=False, percdamp=.01):
    M = M.to(torch.float32)
    H_A = H_A.to(torch.float32)
    H_B = H_B.to(torch.float32)
    W_old = W_old.to(torch.float32)
    
    # 初始化 W 和 Lambda
    W = W_old.clone()
    if lambda_zero:
        Lambda = torch.zeros_like(W)
    # else:
    #     term1 = beta * (torch.mm(W, H_A) - H_B)
    #     term2 = alpha * (W - W_old)
    #     Lambda = -M * (term1 + term2)

    eigvals, Q = torch.linalg.eigh(H_A)

    def apply_A_inv_right(X, eigvals, Q, alpha):
        # X: (n, n)
        d = 1.0 / ((1 - alpha) * eigvals + alpha)  # shape: (n,)
        tmp = X @ Q                                # right multiply Q
        tmp = tmp * d                              # element-wise scale columns
        return tmp @ Q.T                           # right multiply Q^T
    

    for k in tqdm(range(max_iter)):
        alpha = alpha_schedule_exp(k)
        alphas.append(alpha)
        W_prev = W.clone()

        B = (1-alpha) * H_B + alpha * W_prev
        MLambda = Lambda
        X = B - MLambda

        W = apply_A_inv_right(X, eigvals, Q, alpha)

        Lambda = Lambda + rho * (M * W)

        lambda_value.append(Lambda.mean().item())
        lambda_abs.append(Lambda.abs().mean().item())
        out.append((M * W).mean().item())
        # W = W - M * W
        # 收敛判断
        if k % 50 == 0 :
            

            # print( (W.to(torch.float16) @ test_inps[0].cuda() - test_outs[0][0].cuda()).abs().max(), (W.to(torch.float16) @ test_inps[0].cuda() - test_outs[0][0].cuda()).abs().mean())
            # print(torch.norm(W - W_prev))
            if torch.norm(W - W_prev) < epsilon:
                print( (W.to(torch.float16) @ test_inps[0].cuda() - test_outs[0][0].cuda()).abs().max(), (W.to(torch.float16) @ test_inps[0].cuda() - test_outs[0][0].cuda()).abs().mean())
                print(f"Converged at iteration {k}")
                print(torch.norm(W - W_prev))
                break
        # if k % 10 == 0 :
        #     print(torch.norm(W - W_prev))
    W = W - M * W
    return W.to(torch.float16)

test_x = test_inps[0]
test_y = test_outs[0][0]

H_A = torch.zeros((test_x.shape[-2], test_x.shape[-2]), device="cuda")
H_B = torch.zeros((test_y.shape[-2], test_x.shape[-2]), device="cuda")
nsamples = 0
for i in range(120):
    H_A *= nsamples / (nsamples + 1)
    H_B *= nsamples / (nsamples + 1)

    nsamples += 1

    H_A += (math.sqrt(2 / nsamples) * test_inps[i].cuda().to(torch.float32)) @ (math.sqrt(2 / nsamples) * test_inps[i].cuda().to(torch.float32)).T
    H_B += (math.sqrt(2 / nsamples) * test_outs[i][0].cuda().to(torch.float32)) @ (math.sqrt(2 / nsamples) * test_inps[i].cuda().to(torch.float32)).T

    

M = (new_W == 0).to(torch.float32).to(H_B.device)
print(M.device)
# M = torch.zeros_like(M).to(torch.float32).cuda()
print((M == 0).sum() / M.numel())
W_old = new_W.clone().to(H_B.device)
beta = 0.01
alpha = 0.99
gama = 0.0000
rho = 1

start = time.time()

W_update = dual_ascent_method8(H_A, H_B, W_old, M, beta, alpha, gama, rho, lambda_zero=True, max_iter=1000)
print('time', time.time() - start)

cuda:0
tensor(0.3000, device='cuda:0')


 15%|█▌        | 150/1000 [00:04<00:25, 33.38it/s]

tensor(0.5815, device='cuda:0', dtype=torch.float16) tensor(0.0682, device='cuda:0', dtype=torch.float16)
Converged at iteration 150
tensor(0.0222, device='cuda:0')
time 4.870270729064941


In [41]:
print((W_update == 0).sum() / W_update.numel())
for i in [0, 32, 63, 122, 123, 124]:
    # W_update = W_update - M.to(torch.float16) * W_update
    print(test_outs[i][0].abs().max(), test_outs[i][0].abs().mean())
    print(i, (W_update @ test_inps[i].cuda() - test_outs[i][0].cuda()).abs().max(), (W_update @ test_inps[i].cuda() - test_outs[i][0].cuda()).abs().mean())
    print(i, (new_W.to(H_B.device) @ test_inps[i].cuda() - test_outs[i][0].cuda()).abs().max(), (new_W.to(H_B.device) @ test_inps[i].cuda() - test_outs[i][0].cuda()).abs().mean())
    # W_update2 = W_update - M.to(torch.float16) * W_update
    # print(i, (W_update2 @ test_inps[i].cuda() - test_outs[i][0].cuda()).abs().max(), (W_update2 @ test_inps[i].cuda() - test_outs[i][0].cuda()).abs().mean())
    # print("-----------")
    # print((new_W @ test_inps[i]).abs().max(), (new_W @ test_inps[i]).abs().mean())
    # print((W_update2 @ test_inps[i]).abs().max(), (W_update2 @ test_inps[i]).abs().mean())
    # print(test_outs[i][0].abs().max(), test_outs[i][0].abs().mean())
    print("-------------------------------------------------------------------------")

tensor(0.7000, device='cuda:0')
tensor(3.9316, dtype=torch.float16) tensor(0.2715, dtype=torch.float16)
0 tensor(0.5723, device='cuda:0', dtype=torch.float16) tensor(0.0677, device='cuda:0', dtype=torch.float16)
0 tensor(0.5742, device='cuda:0', dtype=torch.float16) tensor(0.0740, device='cuda:0', dtype=torch.float16)
-------------------------------------------------------------------------
tensor(5.1016, dtype=torch.float16) tensor(0.2737, dtype=torch.float16)
32 tensor(0.8359, device='cuda:0', dtype=torch.float16) tensor(0.0694, device='cuda:0', dtype=torch.float16)
32 tensor(0.8164, device='cuda:0', dtype=torch.float16) tensor(0.0758, device='cuda:0', dtype=torch.float16)
-------------------------------------------------------------------------
tensor(3.5605, dtype=torch.float16) tensor(0.2708, dtype=torch.float16)
63 tensor(0.6455, device='cuda:0', dtype=torch.float16) tensor(0.0790, device='cuda:0', dtype=torch.float16)
63 tensor(0.6562, device='cuda:0', dtype=torch.float16) tenso

In [46]:
from tqdm import tqdm
import math
import time

lambda_value = []
lambda_abs = []
alphas = []

out = []

def alpha_schedule_exp(k, max_iter=100, alpha_start=0.9, alpha_end=0.99, gamma=5.0):
    # if k >= max_iter:
    #     return alpha_end
    ratio = k / max_iter
    return alpha_end - (alpha_end - alpha_start) * np.exp(-gamma * ratio)

def delta_schedule_exp(k, max_iter=100, alpha_start=0.9, alpha_end=0.99, gamma=5.0):
    if k >= max_iter:
        return alpha_end - alpha_start
    else:
        return alpha_schedule_exp(k, max_iter, alpha_start, alpha_end, gamma) - alpha_start

def alpha_schedule(k, max_iter=100, alpha_start=0.9, alpha_end=0.99):
    if k >= max_iter:
        return alpha_end
    else:
        return alpha_start + (alpha_end - alpha_start) * (k / max_iter)

def dual_ascent_method10(H_A, H_B, W_old, M, beta, alpha, gama, rho, epsilon=1e-1, max_iter=10, lambda_zero=False, percdamp=.01):
    M = M.to(torch.float32)
    H_A = H_A.to(torch.float32)
    H_B = H_B.to(torch.float32)
    W_old = W_old.to(torch.float32)
    
    # 初始化 W 和 Lambda
    W = W_old.clone()
    if lambda_zero:
        Lambda = torch.zeros_like(W)
    # else:
    #     term1 = beta * (torch.mm(W, H_A) - H_B)
    #     term2 = alpha * (W - W_old)
    #     Lambda = -M * (term1 + term2)
    I = torch.eye(H_A.shape[0], device=H_A.device)
    A = (1-alpha) * H_A + alpha * I

    A_start = torch.linalg.cholesky(A)
    A_start = torch.cholesky_inverse(A_start)
    
    A1 = -(A_start @ (I - H_A) @ A_start)
    A2 = (-A1) @ (I - H_A) @ A_start

    alpha_start = alpha
    for k in tqdm(range(max_iter)):
        alpha = alpha_schedule_exp(k, alpha_start=alpha_start)
        delta = alpha - alpha_start

        if k % 20 == 0 and k <= 100:
            alpha_start = alpha

            A = (1-alpha) * H_A + alpha * I

            A_start = torch.linalg.cholesky(A)
            A_start = torch.cholesky_inverse(A_start)
            
            A1 = -(A_start @ (I - H_A) @ A_start)
            A2 = (-A1) @ (I - H_A) @ A_start



        W_prev = W.clone()

        B = (1-alpha-delta) * H_B + (alpha + delta) * W_prev
        B = B - Lambda

        # A_inv = (1-alpha-delta) * H_A + (alpha + delta) * I
        # A_inv = torch.linalg.cholesky(A_inv)
        # A_inv = torch.cholesky_inverse(A_inv)

        W = B @ (A_start + delta * A1 + delta**2 * A2)
        # W = B @ A_inv

        # print("二阶近似误差",torch.norm(((A_start + delta * A1 + delta**2 * A2) - A_inv)))


        Lambda = Lambda + rho * (M * W)

        lambda_value.append(Lambda.mean().item())
        lambda_abs.append(Lambda.abs().mean().item())
        out.append((M * W).mean().item())
        # W = W - M * W
        # 收敛判断
        if k % 50 == 0 and k > 0:
            

            # print( (W.to(torch.float16) @ test_inps[0].cuda() - test_outs[0][0].cuda()).abs().max(), (W.to(torch.float16) @ test_inps[0].cuda() - test_outs[0][0].cuda()).abs().mean())
            # print(torch.norm(W - W_prev))
            if torch.norm(W - W_prev) < epsilon:
                print( (W.to(torch.float16) @ test_inps[0].cuda() - test_outs[0][0].cuda()).abs().max(), (W.to(torch.float16) @ test_inps[0].cuda() - test_outs[0][0].cuda()).abs().mean())
                print(f"Converged at iteration {k}")
                print(torch.norm(W - W_prev))
                break
        # if k % 10 == 0 :
        #     print(torch.norm(W - W_prev))
    W = W - M * W
    return W.to(torch.float16)

test_x = test_inps[0]
test_y = test_outs[0][0]

H_A = torch.zeros((test_x.shape[-2], test_x.shape[-2]), device="cuda")
H_B = torch.zeros((test_y.shape[-2], test_x.shape[-2]), device="cuda")
nsamples = 0
for i in range(120):
    H_A *= nsamples / (nsamples + 1)
    H_B *= nsamples / (nsamples + 1)

    nsamples += 1

    H_A += (math.sqrt(2 / nsamples) * test_inps[i].cuda().to(torch.float32)) @ (math.sqrt(2 / nsamples) * test_inps[i].cuda().to(torch.float32)).T
    H_B += (math.sqrt(2 / nsamples) * test_outs[i][0].cuda().to(torch.float32)) @ (math.sqrt(2 / nsamples) * test_inps[i].cuda().to(torch.float32)).T

    

M = (new_W == 0).to(torch.float32).to(H_B.device)
print(M.device)
# M = torch.zeros_like(M).to(torch.float32).cuda()
print((M == 0).sum() / M.numel())
W_old = new_W.clone().to(H_B.device)
beta = 0.01
alpha = 0.9
gama = 0.0000
rho = 1


start = time.time()

W_update = dual_ascent_method10(H_A, H_B, W_old, M, beta, alpha, gama, rho, lambda_zero=True, max_iter=1000)
print('time', time.time() - start)

cuda:0
tensor(0.3000, device='cuda:0')


 10%|█         | 100/1000 [00:02<00:22, 39.68it/s]

tensor(0.5752, device='cuda:0', dtype=torch.float16) tensor(0.0674, device='cuda:0', dtype=torch.float16)
Converged at iteration 100
tensor(0.0947, device='cuda:0')
time 2.5368621349334717


In [43]:
from tqdm import tqdm
import math
import time

lambda_value = []
lambda_abs = []
alphas = []

out = []

def alpha_schedule_exp(k, max_iter=100, alpha_start=0.9, alpha_end=0.99, gamma=5.0):
    # if k >= max_iter:
    #     return alpha_end
    ratio = k / max_iter
    return alpha_end - (alpha_end - alpha_start) * np.exp(-gamma * ratio)

def delta_schedule_exp(k, max_iter=100, alpha_start=0.9, alpha_end=0.99, gamma=5.0):
    if k >= max_iter:
        return alpha_end - alpha_start
    else:
        return alpha_schedule_exp(k, max_iter, alpha_start, alpha_end, gamma) - alpha_start

def alpha_schedule(k, max_iter=100, alpha_start=0.9, alpha_end=0.99):
    if k >= max_iter:
        return alpha_end
    else:
        return alpha_start + (alpha_end - alpha_start) * (k / max_iter)

def dual_ascent_method10(H_A, H_B, W_old, M, beta, alpha, gama, rho, epsilon=1e-1, max_iter=10, lambda_zero=False, percdamp=.01):
    M = M.to(torch.float32)
    H_A = H_A.to(torch.float32)
    H_B = H_B.to(torch.float32)
    W_old = W_old.to(torch.float32)
    
    # 初始化 W 和 Lambda
    W = W_old.clone()
    if lambda_zero:
        Lambda = torch.zeros_like(W)
    # else:
    #     term1 = beta * (torch.mm(W, H_A) - H_B)
    #     term2 = alpha * (W - W_old)
    #     Lambda = -M * (term1 + term2)
    I = torch.eye(H_A.shape[0], device=H_A.device)
    A = (1-alpha) * H_A + alpha * I

    A_start = torch.linalg.cholesky(A)
    A_start = torch.cholesky_inverse(A_start)
    
    A1 = -(A_start @ (I - H_A) @ A_start)
    A2 = (-A1) @ (I - H_A) @ A_start
    A3 = (-A2) @ (I - H_A) @ A_start


    alpha_start = alpha
    for k in tqdm(range(max_iter)):
        alpha = alpha_schedule_exp(k, alpha_start=alpha_start)
        delta = alpha - alpha_start

        if k % 20 == 0 and k < 100:
            alpha_start = alpha

            A = (1-alpha) * H_A + alpha * I

            A_start = torch.linalg.cholesky(A)
            A_start = torch.cholesky_inverse(A_start)
            
            A1 = -(A_start @ (I - H_A) @ A_start)
            A2 = (-A1) @ (I - H_A) @ A_start
            A3 = (-A2) @ (I - H_A) @ A_start



        W_prev = W.clone()

        B = (1-alpha-delta) * H_B + (alpha + delta) * W_prev
        B = B - Lambda

        # A_inv = (1-alpha-delta) * H_A + (alpha + delta) * I
        # A_inv = torch.linalg.cholesky(A_inv)
        # A_inv = torch.cholesky_inverse(A_inv)

        W = B @ (A_start + delta * A1 + delta**2 * A2 + delta**3 * A3)
        # W = B @ A_inv

        # print("二阶近似误差",torch.norm(((A_start + delta * A1 + delta**2 * A2) - A_inv)))


        Lambda = Lambda + rho * (M * W)

        lambda_value.append(Lambda.mean().item())
        lambda_abs.append(Lambda.abs().mean().item())
        out.append((M * W).mean().item())
        # W = W - M * W
        # 收敛判断

        if k % 50 == 0 and k > 0:
            

            # print( (W.to(torch.float16) @ test_inps[0].cuda() - test_outs[0][0].cuda()).abs().max(), (W.to(torch.float16) @ test_inps[0].cuda() - test_outs[0][0].cuda()).abs().mean())
            # print(torch.norm(W - W_prev))
            if torch.norm(W - W_prev) < epsilon:
                print( (W.to(torch.float16) @ test_inps[0].cuda() - test_outs[0][0].cuda()).abs().max(), (W.to(torch.float16) @ test_inps[0].cuda() - test_outs[0][0].cuda()).abs().mean())
                print(f"Converged at iteration {k}")
                print(torch.norm(W - W_prev))
                break

        # if k % 10 == 0 :
        #     print(torch.norm(W - W_prev))
    W = W - M * W
    return W.to(torch.float16)

test_x = test_inps[0]
test_y = test_outs[0][0]

H_A = torch.zeros((test_x.shape[-2], test_x.shape[-2]), device="cuda")
H_B = torch.zeros((test_y.shape[-2], test_x.shape[-2]), device="cuda")
nsamples = 0
for i in range(120):
    H_A *= nsamples / (nsamples + 1)
    H_B *= nsamples / (nsamples + 1)

    nsamples += 1

    H_A += (math.sqrt(2 / nsamples) * test_inps[i].cuda().to(torch.float32)) @ (math.sqrt(2 / nsamples) * test_inps[i].cuda().to(torch.float32)).T
    H_B += (math.sqrt(2 / nsamples) * test_outs[i][0].cuda().to(torch.float32)) @ (math.sqrt(2 / nsamples) * test_inps[i].cuda().to(torch.float32)).T

    

M = (new_W == 0).to(torch.float32).to(H_B.device)
print(M.device)
# M = torch.zeros_like(M).to(torch.float32).cuda()
print((M == 0).sum() / M.numel())
W_old = new_W.clone().to(H_B.device)
beta = 0.01
alpha = 0.9
gama = 0.0000
rho = 1


start = time.time()

W_update = dual_ascent_method10(H_A, H_B, W_old, M, beta, alpha, gama, rho, lambda_zero=True, max_iter=1000)
print('time', time.time() - start)

cuda:0
tensor(0.3000, device='cuda:0')


 15%|█▌        | 150/1000 [00:03<00:21, 40.47it/s]

tensor(0.5771, device='cuda:0', dtype=torch.float16) tensor(0.0676, device='cuda:0', dtype=torch.float16)
Converged at iteration 150
tensor(0.0898, device='cuda:0')
time 3.7241785526275635


In [36]:
from tqdm import tqdm
import math
import time

lambda_value = []
lambda_abs = []
alphas = []

out = []

def alpha_schedule_exp(k, max_iter=100, alpha_start=0.9, alpha_end=0.99, gamma=5.0):
    # if k >= max_iter:
    #     return alpha_end
    ratio = k / max_iter
    return alpha_end - (alpha_end - alpha_start) * np.exp(-gamma * ratio)

def delta_schedule_exp(k, max_iter=100, alpha_start=0.9, alpha_end=0.99, gamma=5.0):
    if k >= max_iter:
        return alpha_end - alpha_start
    else:
        return alpha_schedule_exp(k, max_iter, alpha_start, alpha_end, gamma) - alpha_start

def alpha_schedule(k, max_iter=100, alpha_start=0.9, alpha_end=0.99):
    if k >= max_iter:
        return alpha_end
    else:
        return alpha_start + (alpha_end - alpha_start) * (k / max_iter)

def dual_ascent_method10(H_A, H_B, W_old, M, beta, alpha, gama, rho, epsilon=1e-1, max_iter=10, lambda_zero=False, percdamp=.01):
    M = M.to(torch.float32)
    H_A = H_A.to(torch.float32)
    H_B = H_B.to(torch.float32)
    W_old = W_old.to(torch.float32)
    
    # 初始化 W 和 Lambda
    W = W_old.clone()
    if lambda_zero:
        Lambda = torch.zeros_like(W)
    # else:
    #     term1 = beta * (torch.mm(W, H_A) - H_B)
    #     term2 = alpha * (W - W_old)
    #     Lambda = -M * (term1 + term2)
    I = torch.eye(H_A.shape[0], device=H_A.device)
    A = (1-alpha) * H_A + alpha * I

    A_start = torch.linalg.cholesky(A)
    A_start = torch.cholesky_inverse(A_start)
    
    A1 = -(A_start @ (I - H_A) @ A_start)
    A2 = (-A1) @ (I - H_A) @ A_start
    A3 = (-A2) @ (I - H_A) @ A_start
    A4 = (-A3) @ (I - H_A) @ A_start

    alpha_start = alpha
    for k in tqdm(range(max_iter)):
        alpha = alpha_schedule_exp(k, alpha_start=alpha_start)
        delta = alpha - alpha_start

        if k % 20 == 0 and k < 100:
            alpha_start = alpha

            A = (1-alpha) * H_A + alpha * I

            A_start = torch.linalg.cholesky(A)
            A_start = torch.cholesky_inverse(A_start)
            
            A1 = -(A_start @ (I - H_A) @ A_start)
            A2 = (-A1) @ (I - H_A) @ A_start
            A3 = (-A2) @ (I - H_A) @ A_start
            A4 = (-A3) @ (I - H_A) @ A_start


        W_prev = W.clone()

        B = (1-alpha-delta) * H_B + (alpha + delta) * W_prev
        B = B - Lambda

        # A_inv = (1-alpha-delta) * H_A + (alpha + delta) * I
        # A_inv = torch.linalg.cholesky(A_inv)
        # A_inv = torch.cholesky_inverse(A_inv)

        W = B @ (A_start + delta * A1 + delta**2 * A2 + delta**3 * A3 + delta**4 * A4)
        # W = B @ A_inv

        # print("二阶近似误差",torch.norm(((A_start + delta * A1 + delta**2 * A2) - A_inv)))


        Lambda = Lambda + rho * (M * W)

        lambda_value.append(Lambda.mean().item())
        lambda_abs.append(Lambda.abs().mean().item())
        out.append((M * W).mean().item())
        # W = W - M * W
        # 收敛判断

        if k % 50 == 0 and k > 0:
            

            # print( (W.to(torch.float16) @ test_inps[0].cuda() - test_outs[0][0].cuda()).abs().max(), (W.to(torch.float16) @ test_inps[0].cuda() - test_outs[0][0].cuda()).abs().mean())
            # print(torch.norm(W - W_prev))
            if torch.norm(W - W_prev) < epsilon:
                print( (W.to(torch.float16) @ test_inps[0].cuda() - test_outs[0][0].cuda()).abs().max(), (W.to(torch.float16) @ test_inps[0].cuda() - test_outs[0][0].cuda()).abs().mean())
                print(f"Converged at iteration {k}")
                print(torch.norm(W - W_prev))
                break

        # if k % 10 == 0 :
        #     print(torch.norm(W - W_prev))
    W = W - M * W
    return W.to(torch.float16)

test_x = test_inps[0]
test_y = test_outs[0][0]

H_A = torch.zeros((test_x.shape[-2], test_x.shape[-2]), device="cuda")
H_B = torch.zeros((test_y.shape[-2], test_x.shape[-2]), device="cuda")
nsamples = 0
for i in range(120):
    H_A *= nsamples / (nsamples + 1)
    H_B *= nsamples / (nsamples + 1)

    nsamples += 1

    H_A += (math.sqrt(2 / nsamples) * test_inps[i].cuda().to(torch.float32)) @ (math.sqrt(2 / nsamples) * test_inps[i].cuda().to(torch.float32)).T
    H_B += (math.sqrt(2 / nsamples) * test_outs[i][0].cuda().to(torch.float32)) @ (math.sqrt(2 / nsamples) * test_inps[i].cuda().to(torch.float32)).T

    

M = (new_W == 0).to(torch.float32).to(H_B.device)
print(M.device)
# M = torch.zeros_like(M).to(torch.float32).cuda()
print((M == 0).sum() / M.numel())
W_old = new_W.clone().to(H_B.device)
beta = 0.01
alpha = 0.9
gama = 0.0000
rho = 1


start = time.time()

W_update = dual_ascent_method10(H_A, H_B, W_old, M, beta, alpha, gama, rho, lambda_zero=True, max_iter=1000)
print('time', time.time() - start)

cuda:0
tensor(0.3000, device='cuda:0')


 20%|██        | 200/1000 [00:05<00:20, 39.86it/s]

tensor(0.5771, device='cuda:0', dtype=torch.float16) tensor(0.0676, device='cuda:0', dtype=torch.float16)
Converged at iteration 200
tensor(0.0320, device='cuda:0')
time 5.034789562225342


In [37]:
print((W_update == 0).sum() / W_update.numel())
for i in [0, 32, 63, 122, 123, 124]:
    # W_update = W_update - M.to(torch.float16) * W_update
    print(test_outs[i][0].abs().max(), test_outs[i][0].abs().mean())
    print(i, (W_update @ test_inps[i].cuda() - test_outs[i][0].cuda()).abs().max(), (W_update @ test_inps[i].cuda() - test_outs[i][0].cuda()).abs().mean())
    print(i, (new_W.to(H_B.device) @ test_inps[i].cuda() - test_outs[i][0].cuda()).abs().max(), (new_W.to(H_B.device) @ test_inps[i].cuda() - test_outs[i][0].cuda()).abs().mean())
    # W_update2 = W_update - M.to(torch.float16) * W_update
    # print(i, (W_update2 @ test_inps[i].cuda() - test_outs[i][0].cuda()).abs().max(), (W_update2 @ test_inps[i].cuda() - test_outs[i][0].cuda()).abs().mean())
    # print("-----------")
    # print((new_W @ test_inps[i]).abs().max(), (new_W @ test_inps[i]).abs().mean())
    # print((W_update2 @ test_inps[i]).abs().max(), (W_update2 @ test_inps[i]).abs().mean())
    # print(test_outs[i][0].abs().max(), test_outs[i][0].abs().mean())
    print("-------------------------------------------------------------------------")

tensor(0.7000, device='cuda:0')
tensor(3.9316, dtype=torch.float16) tensor(0.2715, dtype=torch.float16)
0 tensor(0.5752, device='cuda:0', dtype=torch.float16) tensor(0.0677, device='cuda:0', dtype=torch.float16)
0 tensor(0.5742, device='cuda:0', dtype=torch.float16) tensor(0.0740, device='cuda:0', dtype=torch.float16)
-------------------------------------------------------------------------
tensor(5.1016, dtype=torch.float16) tensor(0.2737, dtype=torch.float16)
32 tensor(0.8398, device='cuda:0', dtype=torch.float16) tensor(0.0695, device='cuda:0', dtype=torch.float16)
32 tensor(0.8164, device='cuda:0', dtype=torch.float16) tensor(0.0758, device='cuda:0', dtype=torch.float16)
-------------------------------------------------------------------------
tensor(3.5605, dtype=torch.float16) tensor(0.2708, dtype=torch.float16)
63 tensor(0.6489, device='cuda:0', dtype=torch.float16) tensor(0.0790, device='cuda:0', dtype=torch.float16)
63 tensor(0.6562, device='cuda:0', dtype=torch.float16) tenso

In [6]:
print((W_update == 0).sum() / W_update.numel())
for i in [0, 32, 63, 122, 123, 124]:
    # W_update = W_update - M.to(torch.float16) * W_update
    print(test_outs[i][0].abs().max(), test_outs[i][0].abs().mean())
    print(i, (W_update @ test_inps[i].cuda() - test_outs[i][0].cuda()).abs().max(), (W_update @ test_inps[i].cuda() - test_outs[i][0].cuda()).abs().mean())
    print(i, (new_W.to(H_B.device) @ test_inps[i].cuda() - test_outs[i][0].cuda()).abs().max(), (new_W.to(H_B.device) @ test_inps[i].cuda() - test_outs[i][0].cuda()).abs().mean())
    # W_update2 = W_update - M.to(torch.float16) * W_update
    # print(i, (W_update2 @ test_inps[i].cuda() - test_outs[i][0].cuda()).abs().max(), (W_update2 @ test_inps[i].cuda() - test_outs[i][0].cuda()).abs().mean())
    # print("-----------")
    # print((new_W @ test_inps[i]).abs().max(), (new_W @ test_inps[i]).abs().mean())
    # print((W_update2 @ test_inps[i]).abs().max(), (W_update2 @ test_inps[i]).abs().mean())
    # print(test_outs[i][0].abs().max(), test_outs[i][0].abs().mean())
    print("-------------------------------------------------------------------------")

tensor(0.7000, device='cuda:0')
tensor(3.9316, dtype=torch.float16) tensor(0.2715, dtype=torch.float16)
0 tensor(0.5742, device='cuda:0', dtype=torch.float16) tensor(0.0678, device='cuda:0', dtype=torch.float16)
0 tensor(0.5742, device='cuda:0', dtype=torch.float16) tensor(0.0740, device='cuda:0', dtype=torch.float16)
-------------------------------------------------------------------------
tensor(5.1016, dtype=torch.float16) tensor(0.2737, dtype=torch.float16)
32 tensor(0.8359, device='cuda:0', dtype=torch.float16) tensor(0.0695, device='cuda:0', dtype=torch.float16)
32 tensor(0.8164, device='cuda:0', dtype=torch.float16) tensor(0.0758, device='cuda:0', dtype=torch.float16)
-------------------------------------------------------------------------
tensor(3.5605, dtype=torch.float16) tensor(0.2708, dtype=torch.float16)
63 tensor(0.6494, device='cuda:0', dtype=torch.float16) tensor(0.0791, device='cuda:0', dtype=torch.float16)
63 tensor(0.6562, device='cuda:0', dtype=torch.float16) tenso

In [48]:
from tqdm import tqdm
import math
import time
import torch

def alpha_schedule_exp(k, max_iter=100, alpha_start=0.9, alpha_end=0.99, gamma=5.0):
    ratio = min(k / max_iter, 1.0)
    return alpha_end - (alpha_end - alpha_start) * math.exp(-gamma * ratio)

def dual_ascent_method10(H_A, H_B, W_old, M, beta, alpha, gama, rho, epsilon=1e-1, max_iter=1000, lambda_zero=False, percdamp=.01):
    # 数据类型和设备统一
    M = M.to(torch.float32)
    H_A = H_A.to(torch.float32)
    H_B = H_B.to(torch.float32)
    W_old = W_old.to(torch.float32)
    
    # 转换为稀疏矩阵
    M_sparse = M.to_sparse()
    
    # 初始化 W 和 Lambda
    W = W_old.clone()
    if lambda_zero:
        Lambda = torch.zeros_like(W)
    
    # 预计算不变项
    I = torch.eye(H_A.shape[0], device=H_A.device)
    precomputed_term = I - H_A
    
    # 初始矩阵分解
    A_start = torch.linalg.cholesky((1-alpha) * H_A + alpha * I)
    A_start = torch.cholesky_inverse(A_start)
    A1 = -(A_start @ precomputed_term @ A_start)
    A2 = (-A1) @ precomputed_term @ A_start
    
    # 外推和自适应参数
    alpha_threshold = 0.01
    last_alpha = alpha
    lambda_value = []
    lambda_abs = []
    out = []
    
    # Aitken 外推参数
    aitken_freq = 10
    aitken_history = []
    
    # 自适应步长参数
    rho_min = 0.1
    rho_max = 10.0
    rho_adapt_factor = 1.1
    
    for k in tqdm(range(max_iter)):
        # 更新 alpha
        alpha = alpha_schedule_exp(k, alpha_start=alpha)
        delta = alpha - last_alpha
        
        # 仅当 alpha 变化显著时重新计算分解
        if abs(delta) > alpha_threshold:
            A = (1-alpha) * H_A + alpha * I
            A_start = torch.linalg.cholesky(A)
            A_start = torch.cholesky_inverse(A_start)
            A1 = -(A_start @ precomputed_term @ A_start)
            A2 = (-A1) @ precomputed_term @ A_start
            last_alpha = alpha
        
        W_prev = W.clone()
        
        # 主要迭代步骤
        B = (1-alpha-delta) * H_B + (alpha + delta) * W_prev
        B = B - Lambda
        
        # 使用二阶近似计算 A_inv
        W = B @ (A_start + delta * A1 + delta**2 * A2)
        
        # 使用稀疏矩阵更新 Lambda
        Lambda = Lambda + rho * (M_sparse.to_dense() * W)
        
        # 记录指标
        lambda_value.append(Lambda.mean().item())
        lambda_abs.append(Lambda.abs().mean().item())
        out.append((M * W).mean().item())
        
        # # 记录 Aitken 历史
        # aitken_history.append(W.clone())
        # if len(aitken_history) > 3:
        #     aitken_history.pop(0)
        
        # # 应用 Aitken 外推
        # if k % aitken_freq == 0 and len(aitken_history) == 3:
        #     x1, x2, x3 = aitken_history
        #     diff1 = x3 - x2
        #     diff2 = x2 - x1
        #     numerator = torch.norm(diff1, p='fro')**2
        #     denominator = torch.norm(diff1 - diff2, p='fro')
        #     if denominator > 1e-10:
        #         W_aitken = x3 - (numerator / denominator) * diff1
        #         if torch.norm(W_aitken - W_prev) < torch.norm(W - W_prev):
        #             W = W_aitken
        #             aitken_history[-1] = W.clone()
        
        # # 自适应调整 rho
        # if k > 0:
        #     convergence_speed = torch.norm(W - W_prev) / (torch.norm(W_prev) + 1e-10)
        #     if convergence_speed < 0.5:
        #         rho = min(rho * rho_adapt_factor, rho_max)
        #     elif convergence_speed > 0.8:
        #         rho = max(rho / rho_adapt_factor, rho_min)
        
        # 收敛判断
        if k % 50 == 0 and k > 0:
            if torch.norm(W - W_prev) < epsilon:
                print(f"Converged at iteration {k}")
                print(torch.norm(W - W_prev))
                break
    
    W = W - M * W
    return W.to(torch.float16)

# 假设这些变量已经在别处定义
# test_inps, test_outs, new_W

# 计算 H_A 和 H_B
test_x = test_inps[0]
test_y = test_outs[0][0]

H_A = torch.zeros((test_x.shape[-2], test_x.shape[-2]), device="cuda")
H_B = torch.zeros((test_y.shape[-2], test_x.shape[-2]), device="cuda")
nsamples = 0

for i in range(120):
    H_A *= nsamples / (nsamples + 1)
    H_B *= nsamples / (nsamples + 1)
    
    nsamples += 1
    scale = math.sqrt(2 / nsamples)
    x = scale * test_inps[i].cuda().to(torch.float32)
    y = scale * test_outs[i][0].cuda().to(torch.float32)
    
    H_A += x @ x.T
    H_B += y @ x.T

M = (new_W == 0).to(torch.float32).to(H_B.device)
W_old = new_W.clone().to(H_B.device)

# 参数设置
beta = 0.01
alpha = 0.9
gama = 0.0000
rho = 1

# 执行优化后的算法
start = time.time()
W_update = dual_ascent_method10(H_A, H_B, W_old, M, beta, alpha, gama, rho, lambda_zero=True, max_iter=1000)
print('time', time.time() - start)

  5%|▌         | 50/1000 [00:01<00:31, 30.09it/s]

Converged at iteration 50
tensor(0.0230, device='cuda:0')
time 1.7006933689117432


In [49]:
print((W_update == 0).sum() / W_update.numel())
for i in [0, 32, 63, 122, 123, 124]:
    # W_update = W_update - M.to(torch.float16) * W_update
    print(test_outs[i][0].abs().max(), test_outs[i][0].abs().mean())
    print(i, (W_update @ test_inps[i].cuda() - test_outs[i][0].cuda()).abs().max(), (W_update @ test_inps[i].cuda() - test_outs[i][0].cuda()).abs().mean())
    print(i, (new_W.to(H_B.device) @ test_inps[i].cuda() - test_outs[i][0].cuda()).abs().max(), (new_W.to(H_B.device) @ test_inps[i].cuda() - test_outs[i][0].cuda()).abs().mean())
    # W_update2 = W_update - M.to(torch.float16) * W_update
    # print(i, (W_update2 @ test_inps[i].cuda() - test_outs[i][0].cuda()).abs().max(), (W_update2 @ test_inps[i].cuda() - test_outs[i][0].cuda()).abs().mean())
    # print("-----------")
    # print((new_W @ test_inps[i]).abs().max(), (new_W @ test_inps[i]).abs().mean())
    # print((W_update2 @ test_inps[i]).abs().max(), (W_update2 @ test_inps[i]).abs().mean())
    # print(test_outs[i][0].abs().max(), test_outs[i][0].abs().mean())
    print("-------------------------------------------------------------------------")

tensor(0.7000, device='cuda:0')
tensor(3.9316, dtype=torch.float16) tensor(0.2715, dtype=torch.float16)
0 tensor(1.4824, device='cuda:0', dtype=torch.float16) tensor(0.1135, device='cuda:0', dtype=torch.float16)
0 tensor(0.5742, device='cuda:0', dtype=torch.float16) tensor(0.0740, device='cuda:0', dtype=torch.float16)
-------------------------------------------------------------------------
tensor(5.1016, dtype=torch.float16) tensor(0.2737, dtype=torch.float16)
32 tensor(2.2832, device='cuda:0', dtype=torch.float16) tensor(0.1152, device='cuda:0', dtype=torch.float16)
32 tensor(0.8164, device='cuda:0', dtype=torch.float16) tensor(0.0758, device='cuda:0', dtype=torch.float16)
-------------------------------------------------------------------------
tensor(3.5605, dtype=torch.float16) tensor(0.2708, dtype=torch.float16)
63 tensor(1.3125, device='cuda:0', dtype=torch.float16) tensor(0.1188, device='cuda:0', dtype=torch.float16)
63 tensor(0.6562, device='cuda:0', dtype=torch.float16) tenso


=== Testing No Extrapolation ===


 10%|█         | 100/1000 [00:02<00:21, 41.71it/s]


Converged at iteration 100
tensor(0.0918, device='cuda:0')
No Extrapolation took 2.42 seconds

=== Testing Aitken Extrapolation ===


100%|██████████| 1000/1000 [00:21<00:00, 46.42it/s]


Aitken Extrapolation took 21.56 seconds

=== Testing Richardson Extrapolation ===


...

Converged at iteration 100
tensor(0.0346, device='cuda:0')
Richardson Extrapolation took 4.29 seconds

=== Testing Romberg Extrapolation ===


100%|██████████| 1000/1000 [00:43<00:00, 23.15it/s]


Romberg Extrapolation took 43.21 seconds

=== Testing Hybrid Extrapolation ===


 10%|█         | 100/1000 [00:04<00:41, 21.80it/s]


Converged at iteration 100
tensor(0.0346, device='cuda:0')
Hybrid Extrapolation took 4.63 seconds

=== Results Comparison ===
No Extrapolation: Iterations=101, Time=2.42s, Error=396.500000
Aitken Extrapolation: Iterations=1000, Time=21.56s, Error=nan
Richardson Extrapolation: Iterations=101, Time=4.29s, Error=396.250000
Romberg Extrapolation: Iterations=1000, Time=43.21s, Error=nan
Hybrid Extrapolation: Iterations=101, Time=4.63s, Error=396.250000
